## Reconstruct new models without embedding

In [1]:
from megnet.models import MEGNetModel
from megnet.data.graph import GaussianDistance
from megnet.data.crystal import CrystalGraph
import numpy as np

# Pretrained model
model_pretrained = MEGNetModel.from_file('/Users/chenchi/repos/megnet/mvl_models/'
                                         'mp-2018.6.1/band_gap_regression.hdf5')

# New model with length 16 atom feature
cg = CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 100), 0.5), cutoff=4)
model_new = MEGNetModel(100, 2, 16, graph_converter=cg)

Using TensorFlow backend.


In [2]:
# Get pretrained weights
weights = model_pretrained.get_weights()
# The first weight is the element embedding
emb = weights[0] 
# The new model has exact the same layers except the embedding layer
model_new.set_weights(weights[1:]) 

In [3]:
from pymatgen import MPRester

mpr = MPRester()
s = mpr.get_structure_by_material_id('mp-129')

In [4]:
# Pretrained model prediction
model_pretrained.predict_structure(s)

array([0.03076017], dtype=float32)

In [5]:
# Concert manually the atomic number into an atomic feature
graph = model_new.graph_converter.convert(s)
graph['atom'] = [emb[i] for i in graph['atom']]

In [6]:
# Same prediction results
model_new.predict_graph(graph)

array([0.03076017], dtype=float32)